In [1]:
!pip install plotly
!pip install wbdata
!pip install cufflinks
import wbdata
import pandas as pd
import numpy as np
from scipy.optimize import newton

Key '-7578703893284724682' not in persistent cache.
Key '84485591698201752' not in persistent cache.
Key '2895488979215487719' not in persistent cache.
Key '8715403034753921386' not in persistent cache.
Key '-4557832070393809151' not in persistent cache.
Key '790085877682580702' not in persistent cache.
Key '7867721569267061978' not in persistent cache.
Key '-7768019116925203671' not in persistent cache.
Key '6988433700830578119' not in persistent cache.
Key '7151765182697375389' not in persistent cache.
Key '5012770520909437388' not in persistent cache.
Key '-262931279148216788' not in persistent cache.
Key '-3310733041341447756' not in persistent cache.
Key '2055352299715713572' not in persistent cache.
Key '5935416543999780629' not in persistent cache.
Key '6028250528744870180' not in persistent cache.
Key '1793224627627299322' not in persistent cache.
Key '4472502936227412470' not in persistent cache.
Key '4726740348776738433' not in persistent cache.
Key '945543162140439914' not i

# **Population DataFrame** - *Deliverable 3[A]*
This section contains a function that returns a pandas DataFrame indexed by Country, Year and indicator group (including indicators for different age-sex groups). The function allows for dataframes to be created that include multiple countries, a range of years, and multiple indicator groups. Examples can be found below. 

In [29]:
INDICATOR_GROUPS = {
    #Female Population by 5-year age group
    "pop_female_age":{
        'SP.POP.0004.FE': 'Population ages 0-4, female',
        'SP.POP.0509.FE': 'Population ages 5-9, female',
        'SP.POP.1014.FE': 'Population ages 10-14, female',
        'SP.POP.1519.FE': 'Population ages 15-19, female',
        'SP.POP.2024.FE': 'Population ages 20-24, female',
        'SP.POP.2529.FE': 'Population ages 25-29, female',
        'SP.POP.3034.FE': 'Population ages 30-34, female',
        'SP.POP.3539.FE': 'Population ages 35-39, female',
        'SP.POP.4044.FE': 'Population ages 40-44, female',
        'SP.POP.4549.FE': 'Population ages 45-49, female',
        'SP.POP.5054.FE': 'Population ages 50-54, female',
        'SP.POP.5559.FE': 'Population ages 55-59, female',
        'SP.POP.6064.FE': 'Population ages 60-64, female',
        'SP.POP.6569.FE': 'Population ages 65-69, female',
        'SP.POP.7074.FE': 'Population ages 70-74, female',
        'SP.POP.7579.FE': 'Population ages 75-79, female',
        'SP.POP.80UP.FE': 'Population ages 80 and above, female',
    },
    #Male Population by 5-year age group 
    "pop_male_age":{
        'SP.POP.0004.MA': 'Population ages 0-4, male',
        'SP.POP.0509.MA': 'Population ages 5-9, male',
        'SP.POP.1014.MA': 'Population ages 10-14, male',
        'SP.POP.1519.MA': 'Population ages 15-19, male',
        'SP.POP.2024.MA': 'Population ages 20-24, male',
        'SP.POP.2529.MA': 'Population ages 25-29, male',
        'SP.POP.3034.MA': 'Population ages 30-34, male',
        'SP.POP.3539.MA': 'Population ages 35-39, male',
        'SP.POP.4044.MA': 'Population ages 40-44, male',
        'SP.POP.4549.MA': 'Population ages 45-49, male',
        'SP.POP.5054.MA': 'Population ages 50-54, male',
        'SP.POP.5559.MA': 'Population ages 55-59, male',
        'SP.POP.6064.MA': 'Population ages 60-64, male',
        'SP.POP.6569.MA': 'Population ages 65-69, male',
        'SP.POP.7074.MA': 'Population ages 70-74, male',
        'SP.POP.7579.MA': 'Population ages 75-79, male',
        'SP.POP.80UP.MA': 'Population ages 80 and above, male',
    },
    #Young and Old Populations 
    "young_old_pop":{
        'SP.POP.0014.TO': 'Population ages 0-14, total',
        'SP.POP.0014.MA': 'Population ages 0-14, male',
        'SP.POP.0014.FE': 'Population ages 0-14, female',
        'SP.POP.0014.TO.ZS': 'Population ages 0-14 (% of total)',
        'SP.POP.1564.TO': 'Population ages 15-64, total',
        'SP.POP.1564.MA': 'Population ages 15-64, male',
        'SP.POP.1564.FE': 'Population ages 15-64, female',
        'SP.POP.1564.TO.ZS': 'Population ages 15-64 (% of total)',
        'SP.POP.65UP.TO': 'Population ages 65 and above, total',
        'SP.POP.65UP.MA': 'Population ages 65 and above, male',
        'SP.POP.65UP.FE': 'Population ages 65 and above, female',
        'SP.POP.65UP.TO.ZS': 'Population ages 65 and above (% of total)',
    },
    #Rural and Urban Populations 
    "rural_urban_pop":{
        'SP.RUR.TOTL': 'Rural population',
        'SP.RUR.TOTL.ZS': 'Rural population (% of total)',
        'SP.URB.TOTL': 'Urban population',
        'SP.URB.TOTL.ZS': 'Urban population (% of total)',
        'SP.URB.GROW': 'Urban population growth (annual %)',
    },
    #Population Totals 
    "total_pop":{
        'SP.POP.TOTL': 'Population, total',
        'SP.POP.TOTL.FE.ZS': 'Population, female (% of total)',
        'SP.POP.TOTL.MA.ZS': 'Population, male (% of total)',
        'SP.POP.GROW': 'Population growth (annual %)',
    },   
    #Agricultural Data 
    "agriculture": {
        "AG.LND.AGRI.ZS": "Agricultural land (% of land area)",
        "AG.LND.ARBL.ZS": "Arable land (% of land area)",
        "NV.AGR.TOTL.ZS": "Agriculture, forestry, and fishing, value added (% of GDP)",
        "AG.AGR.EMPL.ZS": "Employment in agriculture (% of total employment)",
        "AG.PRD.CREL.KG": "Cereal yield (kg per hectare)",           
        "AG.PRD.FOOD.XD": "Food production index (2014-2016=100)"
    },
    #GDP Data
    "GDP": {
        "NY.GDP.MKTP.CD": "GDP (current US$)",
        "NY.GDP.MKTP.KD.ZG": "GDP growth (annual %)",
        "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
        "NV.IND.TOTL.ZS": "Industry value added (% of GDP)",
        "NV.SRV.TOTL.ZS": "Services value added (% of GDP)",
        "NE.EXP.GNFS.ZS": "Exports of goods and services (% of GDP)",
        "NE.IMP.GNFS.ZS": "Imports of goods and services (% of GDP)",
    }, 
    #Mortality Rates and Life expectancy rates 
    "Mortality_life_expect": {
        "SP.DYN.IMRT.IN": "Mortality rate, infant (per 1,000 live births)",
        "SP.DYN.LE00.IN": "Life expectancy at birth, total (years)",
        "SP.DYN.IMRT.MA.IN": "Mortality rate, infant, male (per 1,000)",
        "SP.DYN.IMRT.FE.IN": "Mortality rate, infant, female (per 1,000)",
        "SP.DYN.LE00.MA.IN": "Life expectancy at birth, male (years)",
        "SP.DYN.LE00.FE.IN": "Life expectancy at birth, female (years)",
    }, 
    #Population Growth, Fertility Rates, and Age Dependacy Ratios
    "demographic_rates": {
        "SP.POP.GROW": "Population growth (annual %)",
        "SP.DYN.TFRT.IN": "Fertility rate, total (births per woman)",
        "SP.ADO.TFRT": "Adolescent fertility rate (births per 1,000 women ages 15-19)",
        "SP.POP.DPND": "Age dependency ratio (% of working-age population)",           
        "SP.POP.DPND.YG": "Age dependency ratio, young (% of working-age population)", 
        "SP.POP.DPND.OL": "Age dependency ratio, old (% of working-age population)", 
    }}

def get_dataframe(countries, years, groups):
    
    #parse countries 
    if isinstance(countries, str): 
        #split by comma or semicolon and strip whitespace 
        if "," in countries:
            country_list = [c.strip() for c in countries.split(',')]
        elif ":" in countries: 
            country_list = [c.strip() for c in countries.split(';')]
        else: 
            #Single Country 
            country_list = [countries.strip()]
     #already a list or single value 
    else: country_list = [countries] if not isinstance(countries, list) else countries

    #parse years 
    if isinstance(years, int): 
        #single year as an integer. example: 2000
        start_year = end_year = years 
    elif isinstance(years, str):
        years = years.strip()
        if "-" in years and not years.startswith("-"):
            #example: "2000-2015"
            start_year, end_year = map(int, years.split("-"))
        elif "to" in years.lower(): 
            #example: "2000 to 2015" 
            parts = years.lower().split("to")
            start_year = int(parts[0].strip())
            end_year = int(parts[1].strip())
        else: 
            #single year as a string. example: "2000" 
            start_year = end_year = int(years)
    else: 
        raise ValueError("Years must be an integer, string year, or year range")

    year_range = (str(start_year), str(end_year))
    
    # parse indicator groups
    if isinstance(groups, str):
        if "," in groups:
            group_list = [g.strip() for g in groups.split(",")]
        elif ";" in groups:
            group_list = [g.strip() for g in groups.split(";")]
        else:
            group_list = [groups.strip()]
    elif isinstance(groups, list):
        group_list = groups
    else:
        raise TypeError(
        "groups must be a string or list of strings "
        "(e.g. 'total_pop' or ['total_pop', 'rural_urban_pop'])"
        )

    for g in group_list:
        if not isinstance(g, str):
            raise TypeError(f"Invalid group type: {type(g)} — expected string")

    #collect all indicator codes from requested groups 
    indicators = {}
    for group in group_list:
        if group not in INDICATOR_GROUPS:
            available = '\n  - '.join(INDICATOR_GROUPS.keys())
            raise ValueError(f"Unknown indicator group '{group}'.\n\nAvailable groups:\n  - {available}")
        indicators.update(INDICATOR_GROUPS[group])

    valid_indicators = {}
    for code, name in indicators.items():
        try:
            test = wbdata.get_series(
                indicator=code,
                country=country_list,
                date=year_range
            )
            if test is not None and len(test) > 0:
                valid_indicators[code] = name
        except Exception:
            pass
    if not valid_indicators:
        raise ValueError("No indicators have data for the selected countries and years.")

    
    data_frame = wbdata.get_dataframe(valid_indicators, country=country_list, date=year_range)

    data_frame = data_frame.sort_index()

    return data_frame 

# Example: Japan and Niger's Female Population Indexed by Age Group in the year 2000

In [37]:
df = get_dataframe("JPN, NER", "2000", "pop_female_age")
df.head()

,"Population ages 0-4, female","Population ages 5-9, female","Population ages 10-14, female","Population ages 15-19, female","Population ages 20-24, female","Population ages 25-29, female","Population ages 30-34, female","Population ages 35-39, female","Population ages 40-44, female","Population ages 45-49, female","Population ages 50-54, female","Population ages 55-59, female","Population ages 60-64, female","Population ages 65-69, female","Population ages 70-74, female","Population ages 75-79, female","Population ages 80 and above, female"
country,,,,,,,,,,,,,,,,,
Japan,2876526.0,2884718.0,3129328.0,3575570.0,4145342.0,4889128.0,4227794.0,3906124.0,3828357.0,4384461.0,5377391.0,4370492.0,4015631.0,3747472.0,3237474.0,2560664.0,3300344.0
Niger,1156941.0,839232.0,681188.0,577228.0,478574.0,393501.0,331340.0,280624.0,231528.0,188413.0,158129.0,135695.0,99901.0,69942.0,44322.0,23840.0,11862.0


# Example 2: Niger's Mortality and Life Expectancy over 10 years 

In [38]:
df_two = get_dataframe("NER", "1965-1975", "Mortality_life_expect")
df_two.head()

,"Mortality rate, infant (per 1,000 live births)","Life expectancy at birth, total (years)","Mortality rate, infant, male (per 1,000)","Mortality rate, infant, female (per 1,000)","Life expectancy at birth, male (years)","Life expectancy at birth, female (years)"
date,,,,,,
1965,NaN,36.404,NaN,NaN,35.837,36.870
1966,NaN,36.331,NaN,NaN,35.784,36.774
1967,NaN,36.243,NaN,NaN,35.698,36.684
1968,NaN,36.128,NaN,NaN,35.585,36.568
1969,149.5,36.003,158.8,139.6,35.444,36.462
